Привет, меня зовут Артем Хуршудов. Сегодня я проверю твой проект.
<br> Дальнейшее общение будет происходить на "ты" если это не вызывает никаких проблем.
<br> Желательно реагировать на каждый мой комментарий ('исправил', 'не понятно как исправить ошибку', ...)
<br> Пожалуйста, не удаляй комментарии ревьюера, так как они повышают качество повторного ревью.

Комментарии будут в <font color='green'>зеленой</font>, <font color='blue'>синей</font> или <font color='red'>красной</font> рамках:

<div class="alert alert-block alert-success">
<b>Успех:</b> Если все сделано отлично
</div>

<div class="alert alert-block alert-info">
<b>Совет: </b> Если можно немного улучшить
</div>

<div class="alert alert-block alert-danger">
<b>Ошибка:</b> Если требуются исправления. Работа не может быть принята с красными комментариями.
</div>

-------------------

Будет очень хорошо, если ты будешь помечать свои действия следующим образом:
<div class="alert alert-block alert-warning">
<b>Комментарий студента:</b> ...
</div>

<div class="alert alert-block alert-warning">
<b>Изменения:</b> Были внесены следующие изменения ...
</div>

### <font color='orange'>Общее впечатление</font>
* Большое спасибо за проделанную работу. Видно, что приложено много усилий.
* Выводы и рассуждения получились содержательными, их было интересно читать.
* Отлично, что доверительный интервал и риск убытков посчитаны верно.
* Над этим проектом нужно будет еще немного поработать. Однако, изменения не должны занять много времени.
* В работе я оставил несколько советов. Буду рад, если ты учтешь их.
* С радостью отвечу на твои вопросы, если они есть. Лучше всего их собрать в следующей ячейке. Жду новую версию проекта.

### <font color='orange'>Общее впечатление (ревью 2)</font>
* После исправлений проект улучшился и теперь он может быть зачтен.
* Для удобства все новые комментарии обозначены фразой "ревью 2".
* Удачи в дальнейшем обучении и следующих работах!

# Выбор локации для скважины

Допустим, вы работаете в добывающей компании «ГлавРосГосНефть». Нужно решить, где бурить новую скважину.

Вам предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Постройте модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализируйте возможную прибыль и риски техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

## Загрузка и подготовка данных

 ###  Подключение библиотек

In [1]:
import pandas as pd
import math 
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

<div class="alert alert-block alert-success">
<b>Успех:</b> Отлично, что все импорты собраны в первой ячейке ноутбука! Если у того, кто будет запускать твой ноутбук будут отсутствовать некоторые библиотеки, то он это увидит сразу, а не в процессе!
</div>

 ###  Загрузка файла

In [2]:
try:
    data = pd.read_csv('C:/Users/elve/Desktop/Обучение/Churn_Modelling.csv')
except:
    data_0 = pd.read_csv('/datasets/geo_data_0.csv')
    data_1 = pd.read_csv('/datasets/geo_data_1.csv')
    data_2 = pd.read_csv('/datasets/geo_data_2.csv')
print(data_0.columns)
print(data_1.columns)
print(data_2.columns)    

Index(['id', 'f0', 'f1', 'f2', 'product'], dtype='object')
Index(['id', 'f0', 'f1', 'f2', 'product'], dtype='object')
Index(['id', 'f0', 'f1', 'f2', 'product'], dtype='object')


 ###  Проверка данных
 * Проверим на дубликаты
 * Проверим на пропуски данных
 * Проверим верность типов данных

In [3]:
def check_data (data_i):
    temp = data_i.copy() 
# проверяем дубликаты
    print('Кол-во дубликатов:',temp.duplicated().sum())
    list_c=['id', 'f0', 'f1', 'f2', 'product']
    print(temp.info())
    for col_l in list_c:
        print('-------------------------')
        print(col_l, temp[col_l].sort_values().unique())
        print(col_l,': кол-во NaN',temp[col_l].isna().sum(),
        ', процент NaN', round(temp[col_l].isna().sum()/len(temp)*100, 2),'%')

In [4]:
check_data (data_0)

Кол-во дубликатов: 0
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
None
-------------------------
id ['006OJ' '009eY' '00AfQ' ... 'zztWK' 'zzyhQ' 'zzzLH']
id : кол-во NaN 0 , процент NaN 0.0 %
-------------------------
f0 [-1.40860531 -1.35177299 -1.30222711 ...  2.33375269  2.33707957
  2.36233081]
f0 : кол-во NaN 0 , процент NaN 0.0 %
-------------------------
f1 [-0.8482185  -0.84490792 -0.8205609  ...  1.33334561  1.33482762
  1.34376933]
f1 : кол-во NaN 0 , процент NaN 0.0 %
-------------------------
f2 [-12.08832812 -10.13834135 -10.13817115 ...  15.23032159  15.42837187
  16.00379001]
f2 : кол-во N

In [5]:
check_data (data_1)

Кол-во дубликатов: 0
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
None
-------------------------
id ['0022J' '003Gl' '003Vx' ... 'zzv4E' 'zzy2c' 'zzzvI']
id : кол-во NaN 0 , процент NaN 0.0 %
-------------------------
f0 [-31.60957602 -27.82961614 -26.64625507 ...  28.93082879  29.25906208
  29.42175461]
f0 : кол-во NaN 0 , процент NaN 0.0 %
-------------------------
f1 [-26.35859801 -25.38962242 -25.2915177  ...  16.0268693   16.7371962
  18.73406263]
f1 : кол-во NaN 0 , процент NaN 0.0 %
-------------------------
f2 [-0.01814409 -0.01788668 -0.01768626 ...  5.01750345  5.01909142
  5.01972056]
f2 : кол

In [6]:
check_data (data_2)

Кол-во дубликатов: 0
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
None
-------------------------
id ['009Gl' '00AuD' '00CaL' ... 'zzqqy' 'zzsKd' 'zzz9h']
id : кол-во NaN 0 , процент NaN 0.0 %
-------------------------
f0 [-8.76000362 -7.45058711 -7.18949804 ...  7.19461485  7.21552717
  7.23826248]
f0 : кол-во NaN 0 , процент NaN 0.0 %
-------------------------
f1 [-7.08401976 -6.74835677 -6.73299712 ...  7.10161842  7.76185714
  7.84480127]
f1 : кол-во NaN 0 , процент NaN 0.0 %
-------------------------
f2 [-11.97033454 -11.61169048 -11.40724351 ...  16.31301122  16.35764509
  16.73940206]
f2 : кол-во N

In [7]:
data_0=data_0.drop('id', axis=1)
data_1=data_1.drop('id', axis=1)
data_2=data_2.drop('id', axis=1)

<b>Вывод:</b> Данные не имеют пропусков и дубликатов. Убран столбец ID так как не влияет на результаты модели.

<div class="alert alert-block alert-success">
<b>Успех:</b> Данные загружены и осмотрены корректно. Согласен, что колонка id нам не нужна.
</div>

## Обучение и проверка модели

In [16]:
def change_reg (data_i):
    target = data_i['product']
    features = data_i.drop('product', axis=1)
    features_train, features_valid,target_train, target_valid = train_test_split(features,target, train_size=0.75, random_state=12345)
    model = LinearRegression()
    model.fit(features_train,target_train)
    predicted_valid = model.predict(features_valid)
    data = {'target': target_valid, 'predict': predicted_valid}
    df = pd.DataFrame(data, columns = ['target', 'predict'] )
    mean_predict=predicted_valid.mean()
    sum_predict=sum(predicted_valid)
    mse = mean_squared_error(target_valid, predicted_valid)
    rmse = math.sqrt(mse)

    return df,mean_predict,sum_predict,mse,rmse

In [17]:

df_0,mp_0,sp_0,mse_0,rmse_0=change_reg (data_0)
print ("Результат по региону 0:")
print('Cредний запас предсказанного сырья  = {:.2f}'.format(mp_0))
print('Общий запас предсказанного сырья  = {: ,.2f}'.format(sp_0))
print('RSME = {:.2f}'.format(rmse_0))
print(df_0.head(3))
print()

df_1,mp_1,sp_1,mse_1,rmse_1=change_reg (data_1)
print ("Результат по региону 1:")
print('Cредний запас предсказанного сырья  = {:.2f}'.format(mp_1))
print('Общий запас предсказанного сырья  = {: ,.2f}'.format(sp_1))
print('RSME = {:.2f}'.format(rmse_1))
print(df_1.head(3))
print()

df_2,mp_2,sp_2,mse_2,rmse_2=change_reg (data_2)
print ("Результат по региону 2:")
print('Cредний запас предсказанного сырья  = {:.2f}'.format(mp_2))
print('Общий запас предсказанного сырья  = {: ,.2f}'.format(sp_2))
print('RSME = {:.2f}'.format(rmse_2))
print(df_2.head(3))
print()

Результат по региону 0:
Cредний запас предсказанного сырья  = 92.59
Общий запас предсказанного сырья  =  2,314,814.19
RSME = 37.58
           target    predict
71751   10.038645  95.894952
80493  114.551489  77.572583
2655   132.603635  77.892640

Результат по региону 1:
Cредний запас предсказанного сырья  = 68.73
Общий запас предсказанного сырья  =  1,718,213.67
RSME = 0.89
          target    predict
71751  80.859783  82.663314
80493  53.906522  54.431786
2655   30.132364  29.748760

Результат по региону 2:
Cредний запас предсказанного сырья  = 94.97
Общий запас предсказанного сырья  =  2,374,126.15
RSME = 40.03
          target    predict
71751  61.212375  93.599633
80493  41.850118  75.105159
2655   57.776581  90.066809



<b>Вывод:</b> Для каждого региона проведено моделирования целевого показателя [product] с помощью модели линейной регрессии.
Полученные показатели RSMEв случае 0 и 2 региона имеет допустимое значение (40 и 37).  Результат RSME 1 региона стримиться к 0, но при этом в регионе наименьшие средние запасы нефти. 


<div class="alert alert-block alert-success">
<b>Успех:</b> Модели обучены корректно. Радует, что ты активно используешь функции. Это помогает избежать дублирования кода, а значит уменьшает вероятность совершить ошибку/опечатку!
</div>

<div class="alert alert-block alert-info">
<b>Совет: </b> Внутри функции переменные лучше называть маленькими буквами, так как это не константы.
</div>

<div class="alert alert-block alert-warning">
<b>Изменения:</b> Убрал большие буквы
</div>

<div class="alert alert-block alert-success">
<b>Успех (ревью 2):</b> ОК.
</div>

## Подготовка к расчёту прибыли

In [18]:
INVEST_COST=10000000000
ALL_POINT=500
BEST_POINT=200
REV_FOR_1BARL=450000
avg_rev_for_1point=INVEST_COST/BEST_POINT
avg_product_for_1point=avg_rev_for_1point/REV_FOR_1BARL
print('Минимальное среднее количество продукта = {:.2f}'.format(avg_product_for_1point))
print('Cреднее количество продукта в регионе 0 = {:.2f}'.format(data_0['product'].mean()))
print('Cреднее количество продукта в регионе 1 = {:.2f}'.format(data_1['product'].mean()))
print('Cреднее количество продукта в регионе 2 = {:.2f}'.format(data_2['product'].mean()))



Минимальное среднее количество продукта = 111.11
Cреднее количество продукта в регионе 0 = 92.50
Cреднее количество продукта в регионе 1 = 68.83
Cреднее количество продукта в регионе 2 = 95.00


<div class="alert alert-block alert-info">

<b>Совет: </b> В названиях константных переменных лучше использовать только БОЛЬШИЕ буквы. Это соглашение между программистами: https://pythonworld.ru/osnovy/pep-8-rukovodstvo-po-napisaniyu-koda-na-python.html#id29 .
</div>

<div class="alert alert-block alert-warning">
<b>Изменения:</b> Перевел  константы в верхний регистр
</div>

<div class="alert alert-block alert-success">
<b>Успех (ревью 2):</b> Спасибо!
</div>

<b>Выводы:</b>  
    Хотя среднее значение запаса сырья по регионом меньше, чем минимально допустимое, мы видим, что регионы 1 и 3 совсем немного не добирают, следовательно при дальнейшей выборке наилучших месторождений можно прогнозировать запас выше минимального среднего. Для региона 2 так же сохраняется вероятность обеспечить средней запас выше минимального значения. 


In [11]:
def d(data_t,data_p):
    data_p=data_p.sort_values(ascending=False)
    data=data_t[data_p.index].head(200)
    all_rev_in_top200 =sum(data)*rev_for_1barl-invest_cost
    barl_in_top200= data.mean()
   # print('Cреднее количество продукта = {:.2f}'.format(data_i.mean()))
   # print('Cреднее количество продукта(топ 200) = {:.2f}'.format(barl_in_top200))
   # print('Прибыль = ${:,.2f}'.format(  all_rev_in_top200))
    return all_rev_in_top200

<div class="alert alert-block alert-danger">
<b>Ошибка:</b> Для выбора 200 лучших скважин нужно использовать предсказанные значения. А вот для подсчета прибыли должны использоваться настоящие значения соответствующих ячеек.
</div>

<div class="alert alert-block alert-warning">
<b>Изменения:</b> Добавил индексы из  выборки предсказаний.
</div>

<div class="alert alert-block alert-success">
<b>Успех (ревью 2):</b> Верно.
</div>

## Расчёт прибыли и рисков 

In [12]:

state = np.random.RandomState(12345)
def d2(target,predict):
    values = []
    negativ=0
    for i in range(1000):
        target_subsample = target.sample(n=all_point, replace=True, random_state=state)
        predict_subsample= predict[target_subsample.index]
        rezult =d(target_subsample,predict_subsample)
        if rezult<0:
            negativ+=1
        values.append(rezult)
    values=pd.Series(values)
    lower =  values.quantile(0.025)
    upper =  values.quantile(0.975)
    mean = values.mean()
    risk=negativ/1000*100
    print('Cредняя прибыль  = ${:,.2f}'.format(mean))
    print(' 95%-й доверительный интервал  от ${:,.2f} до ${:,.2f} тыс.руб.'.format(lower,upper))
    print('Риск убытка от {:,.1f}%'.format(risk) )
    print()
    return
  


<div class="alert alert-block alert-success">
<b>Успех:</b> Отлично, что для подсчета доверительного интервала использовался именно метод квантилей!
</div>


<div class="alert alert-block alert-info">

<b>Совет: </b> Риск можно было найти чуть проще: `(values < 0).mean()`, если объект типа pandas Series или numpy array.
</div>

In [14]:
  print ("Результат по региону 0:")
d2(df_0['target'],df_0['predict'])
  print ("Результат по региону 1:")
d2(df_1['target'],df_1['predict'])
  print ("Результат по региону 2:")
d2(df_2['target'],df_2['predict'])


Результат по региону 0:
Cредняя прибыль  = $425,938,526.91
 95%-й доверительный интервал  от $-102,090,094.84 до $947,976,353.36 тыс.руб.
Риск убытка от 6.0%

Результат по региону 1:
Cредняя прибыль  = $518,259,493.70
 95%-й доверительный интервал  от $128,123,231.43 до $953,612,982.07 тыс.руб.
Риск убытка от 0.3%

Результат по региону 2:
Cредняя прибыль  = $420,194,005.34
 95%-й доверительный интервал  от $-115,852,609.16 до $989,629,939.84 тыс.руб.
Риск убытка от 6.2%



<s><b>Выводы</b> Регионы 0 и 2 остаются наиболее перспективными для разработки.  Регион 1 также показывает возможность получения прибыли, но возникают риски убытка и уровень прибыли ниже относительно двух других регионов. 
Выбираем регион 2</s>

<div class="alert alert-block alert-warning">
<b>Выводы:</b> Результаты повтроного анализа (после исправления ошибки) приводят к тому, что только регион 1 окупит  инвестициию. По региону 0 и 2 риски убытка высокие. Выбираем регион №1 
</div>

<div class="alert alert-block alert-success">
<b>Успех (ревью 2):</b> Теперь все статистики посчитаны верно. С выбором региона согласен.
</div>

## Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  Jupyter Notebook открыт
- [x]  Весь код выполняется без ошибок
- [x]  Ячейки с кодом расположены в порядке исполнения
- [x]  Выполнен шаг 1: данные подготовлены
- [x]  Выполнен шаг 2: модели обучены и проверены
    - [X]  Данные корректно разбиты на обучающую и валидационную выборки
    - [X]  Модели обучены, предсказания сделаны
    - [X]  Предсказания и правильные ответы на валидационной выборке сохранены
    - [X]  На экране напечатаны результаты
    - [X]  Сделаны выводы
- [ ]  Выполнен шаг 3: проведена подготовка к расчёту прибыли
    - [X]  Для всех ключевых значений созданы константы Python
    - [X]  Посчитано минимальное среднее количество продукта в месторождениях региона, достаточное для разработки
    - [X]  По предыдущему пункту сделаны выводы
    - [X]  Написана функция расчёта прибыли
- [ ]  Выполнен шаг 4: посчитаны риски и прибыль
    - [X]  Проведена процедура *Bootstrap*
    - [X]  Все параметры бутстрепа соответствуют условию
    - [X]  Найдены все нужные величины
    - [x]  Предложен регион для разработки месторождения
    - [x]  Выбор региона обоснован